References:
https://keras.io/examples/timeseries/timeseries_anomaly_detection/
https://huggingface.co/keras-io/time-series-anomaly-detection-autoencoder
https://curiousily.com/posts/anomaly-detection-in-time-series-with-lstms-using-keras-in-python/
https://valueml.com/anomaly-detection-in-time-series-data-using-keras/
https://towardsdatascience.com/time-series-of-price-anomaly-detection-with-lstm-11a12ba4f6d9